In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import anndata
import string
import gc
from anndata import read_h5ad
from anndata import read_csv
from pandas import DataFrame
import h5py
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import networkx as nx
import plotly.graph_objects as go
import scanpy as sc
from cnmf import cNMF
import pickle

/home/anton.thieme/miniconda3/envs/sc_env_cnmf/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)


## PREPARE DATA

In [2]:
adata_tf = read_h5ad(os.path.join('..', 'data', 'adata_tf.h5ad'))
adata_tf

AnnData object with n_obs × n_vars = 1093048 × 1637
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'method_colors

In [3]:
sc.pp.filter_cells(adata_tf, min_genes=3) # filter cells with fewer than 3 genes
#sc.pp.filter_cells(adata_tf, min_counts=200)  # This is a weaker threshold than above. It is just to population the n_counts column in adata
sc.pp.filter_genes(adata_tf, min_cells=3) # filter genes detected in fewer than 3 cells

In [4]:
sc.write(os.path.join('..', 'data', 'adata_tf_cnmf.h5ad'), adata_tf)

## RUN CNMF

follow tutorial from here: https://github.com/dylkot/cNMF/blob/master/Tutorials/analyze_pbmc_example_data.ipynb

In [7]:
numiter=20 # Number of NMF replicates. Set this to a larger value ~200 for real data. We set this to a relatively low value here for illustration at a faster speed
numhvgenes=2000 ## Number of over-dispersed genes to use for running the actual factorizations

## Results will be saved to [output_directory]/[run_name] which in this example is example_PBMC/cNMF/pbmc_cNMF
output_directory = '../results/cnmf'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
run_name = 'tf_cNMF'

## Specify the Ks to use as a space separated list in this case "5 6 7 8 9 10"
K = ' '.join([str(i) for i in range(5,20)])

## To speed this up, you can run it for only K=7-8 with the option below
#K = ' '.join([str(i) for i in range(7,9)])


seed = 14 ## Specify a seed pseudorandom number generation for reproducibility

## Path to the filtered counts dataset we output previously
countfn = '../data/adata_tf_cnmf.h5ad'

In [8]:
## Initialize the cnmf object that will be used to run analyses
cnmf_obj = cNMF(output_dir=output_directory, name=run_name)

In [9]:
## Prepare the data, I.e. subset to 2000 high-variance genes, and variance normalize
cnmf_obj.prepare(counts_fn=countfn, components=np.arange(5,11), n_iter=20, seed=14, num_highvar_genes=2000)

# Pickle the cNMF object
with open(os.path.join(output_directory, 'cnmf_obj_after_prepare.pkl'), 'wb') as f:
    pickle.dump(cnmf_obj, f)

/home/anton.thieme/miniconda3/envs/sc_env_cnmf/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [ ]:
## Specify that the jobs are being distributed over a single worker (total_workers=1) and then launch that worker
cnmf_obj.factorize(worker_i=0, total_workers=1)

with open(os.path.join(output_directory, 'cnmf_obj_after_factorize.pkl'), 'wb') as f:
    pickle.dump(cnmf_obj, f)

[Worker 0]. Starting task 0.
[Worker 0]. Starting task 1.
[Worker 0]. Starting task 2.
[Worker 0]. Starting task 3.
[Worker 0]. Starting task 4.
[Worker 0]. Starting task 5.
[Worker 0]. Starting task 6.
[Worker 0]. Starting task 7.
[Worker 0]. Starting task 8.
[Worker 0]. Starting task 9.
[Worker 0]. Starting task 10.
[Worker 0]. Starting task 11.
[Worker 0]. Starting task 12.
[Worker 0]. Starting task 13.
[Worker 0]. Starting task 14.
[Worker 0]. Starting task 15.


In [ ]:
cnmf_obj.combine()

In [2]:
adata_tf = read_h5ad(os.path.join('..', 'data', 'adata_tf.h5ad'))
adata_tf

AnnData object with n_obs × n_vars = 1093048 × 1637
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'method_colors

In [ ]:
# Export the count matrix (cells x genes)
adata_tf.to_df().to_csv('../results/cnmf/expression_matrix.csv')

# Export the gene names
pd.DataFrame({'gene': adata_tf.var_names}).to_csv('../results/cnmf/gene_names.csv', index=False)

In [7]:
# Initialize cNMF object
output_dir = "../results/cnmf"
os.makedirs(output_dir, exist_ok=True)

cnmf_obj = cNMF(output_dir=output_dir)

# Prepare data
cnmf_obj.prepare(
    counts_fn=os.path.join(output_dir, 'expression_matrix.csv'),
    genes_file=os.path.join(output_dir, 'gene_names.csv'),
    components=np.arange(5, 14),
    n_iter=10,
    seed=42
)

# Perform factorization
cnmf_obj.factorize(
    components=10,  # Adjust number of components as needed
    n_iter=10,  # Number of iterations for robust results
    random_state=42  # Ensures reproducibility
)

# Calculate consensus
cnmf_obj.consensus()

/home/anton.thieme/miniconda3/envs/sc_env_cnmf/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


KeyError: "Values ['gene', 'NFYA', 'ARX', 'HOXA11', 'MEOX1', 'SOX8', 'ZNF195', 'ZFX', 'YBX2', 'ZNF263', 'DLX6', 'ETV1', 'GTF2IRD1', 'PAX6', 'TEAD3', 'E2F2', 'TFAP2B', 'TFAP2D', 'NFIX', 'PAX7', 'MLXIPL', 'ETV7', 'ZNF207', 'ZNF200', 'SCMH1', 'HIVEP2', 'AKAP8L', 'DPF1', 'WIZ', 'ZBTB32', 'LTF', 'NR1H4', 'ISL1', 'ZNF582', 'SNAI2', 'ZFP64', 'RUNX3', 'HSF2', 'PHF20', 'NR1H3', 'POU2F2', 'TBPL1', 'ARNTL2', 'IKZF2', 'ZZZ3', 'HOXC8', 'SOX30', 'THAP3', 'BARX2', 'ZIC2', 'ZNF800', 'FOXP3', 'NFE2L3', 'ALX4', 'FOXN3', 'FOXC1', 'ZNF280C', 'PRDM1', 'FLYWCH1', 'MXD1', 'YBX3', 'CREB3L3', 'PRDM6', 'GPBP1', 'ZNF112', 'AHRR', 'GSC2', 'ZNF275', 'DLX3', 'DMRT3', 'BORCS8-MEF2B', 'POU1F1', 'HMG20B', 'ZNF76', 'SPEN', 'FOXJ2', 'YBX1', 'NFYC', 'SPI1', 'ZBTB11', 'MSANTD3', 'ZFAT', 'SP100', 'KLF6', 'ZFY', 'MEF2A', 'TFE3', 'PITX1', 'RORA', 'HES2', 'MNT', 'ZXDC', 'TCF3', 'MBD3', 'SREBF1', 'NFATC3', 'TP63', 'TBX21', 'GLI2', 'TEAD2', 'ZNF532', 'ZNF37A', 'FOSL2', 'PAX2', 'BAZ2A', 'RARB', 'NFKB2', 'HOXA9', 'TP73', 'SP140',